# TL;DR – Too Long, Doctor

TL;DR is a ML model designed to synthesize and cluster scientific papers. Tailored for both students and researchers seeking to optimize their study time, TL;DR provides a tool to quickly grasp the essence of complex scientific material. Additionally, it caters to those who desire a concise summary or a preliminary overview of a paper before delving into a detailed reading.

# Importing libraries

In [ ]:
# Import library to extract data from XML file
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os

In [ ]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
import torch
from transformers import BertTokenizer

# Global Functions

In [ ]:
"""
    Extracts data from an XML file and returns it as a dictionary.

    Args:
        file (str): The path to the XML file.

    Returns:
        dict: A dictionary containing the extracted data.
"""
def extract_data(file):
    # Create a dictionary to store the data
    data = {}
    # Parse the XML file
    tree = ET.parse(file)
    # Get the root of the XML file
    root = tree.getroot()

    # Initialize abstract data
    data['abstract'] = {}

    # Initialize body data
    data['body'] = []

    # Initialize keywords data
    data['keywords'] = []

    # Extract title and abstract
    article_meta = root.find('.//article-meta')
    if article_meta is not None:
        title_group = article_meta.find('title-group')
        data['title'] = title_group.find('article-title').text if title_group is not None else None

        abstract_section = article_meta.find('abstract')
        if abstract_section is not None:
            for section in abstract_section.findall('sec'):
                section_title = section.find('title').text if section.find('title') is not None else ''
                section_text = section.find('p').text if section.find('p') is not None else ''
                if 'simple summary' in section_title.lower():
                    data['abstract']['simple_summary'] = section_text
                elif 'abstract' in section_title.lower():
                    data['abstract']['abstract'] = section_text

        # Extract keywords
        kwd_group = article_meta.find('kwd-group')
        if kwd_group is not None:
            data['keywords'] = [kwd.text for kwd in kwd_group.findall('kwd') if kwd.text]

    # Extract body sections
    body_section = root.find('body')
    if body_section is not None:
        for sec in body_section.findall('sec'):
            section_data = {
                'title': sec.find('title').text if sec.find('title') is not None else None,
                'content': [p.text for p in sec.findall('p') if p.text]
            }
            data['body'].append(section_data)

    # Return the extracted data
    return data

# Dataset Generation

In [ ]:
# Extract data from the XML files
# Create a list to store the data
data = []
# Get the path of the XML files
path = './data'

# Get the list of the XML files
files = os.listdir(path)

# Loop through the XML files
for file in files:
    # Extract data from the XML file
    data.append(extract_data(path + '/' + file))

In [ ]:
# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(data)
# Save the DataFrame as a JSON file
df.to_json('data.json', orient='records')

In [ ]:
# iterate over data list to print its contents
for i in range(len(data)):
    # print the title of the article
    print(data[i]['title'])

In [ ]:
for i in range(len(data)):
    # print the abstract of the article
    print(data[i]['abstract'])

In [ ]:
for i in range(len(data)):
    # print the body of the article
    print(data[i]['body'])

In [ ]:
# Entra dentro body per combinare tutti i paragrafi in un unico testo
# Cicla su tutti i dizonari dentro body, che sono i paragrafi
# I paragrafi sono un dizionario con chiave title e content
# Title contiene una stringa con il titolo del paragrafo
# Content è una lista di stringhe che vanno combinate in un unico testo
# Cicla su content e combina tutte le stringhe in un unico testo

for i in range(len(data)):
    # print each element of body inside data
    for section in data[i]["body"]:
        # print the title of the section
        print(section["title"])
        # print the content of the section
        print(section["content"])

In [ ]:
def combine_body_content(body_list):
    combined_content = []

    # Verifica che il body sia una lista
    if isinstance(body_list, list):
        # Cicla su tutti i dizionari dentro body, che sono i paragrafi
        for section in body_list:
            # Ottieni il titolo e il contenuto della sezione, se esistente
            title = section.get('title')
            content = ' '.join(section.get('content', []))
            # Combina il titolo e il contenuto con uno spazio e aggiungi al contenuto combinato
            combined_section = ' '.join(filter(None, [title, content])).strip()
            combined_content.append(combined_section)
    # Unisci tutte le sezioni in una singola stringa separata da spazi
    return ' '.join(combined_content)

# Applica la funzione alla colonna "body" del dataframe
df['combined_body'] = df['body'].apply(combine_body_content)

In [ ]:
# Verifica il contenuto della nuova colonna "combined_body"
print(df['combined_body'].head())

In [ ]:
for i in range(len(data)):
    # print the keywords of the article
    print(data[i]['keywords'])

In [ ]:
print(type(df.loc[0, 'abstract']))

In [ ]:
print(df.dtypes)

In [ ]:
# Funzione per combinare "simple summary" e "abstract" gestendo i valori None
def combine_abstract(abstract):
    if isinstance(abstract, dict):
        simple_summary = abstract.get('simple_summary') or ''  # Restituisce una stringa vuota se il valore è None
        abstract_text = abstract.get('abstract') or ''  # Restituisce una stringa vuota se il valore è None
        return ' '.join([simple_summary, abstract_text]).strip()
    return ''

# Applica la funzione a ciascuna riga della colonna "abstract"
df['combined_abstract'] = df['abstract'].apply(combine_abstract)

# Verifica il risultato
print(df['combined_abstract'].head())

In [ ]:
print(df.dtypes)

In [ ]:
# Stampa i primi elementi della colonna "combined_abstract"
print(df['combined_abstract'].head())

# Verifica il tipo del primo elemento della colonna "combined_abstract"
print(type(df.loc[0, 'combined_abstract']))

# Text Classification

For the Text Classification task we will use BERT.

## Pre-Processing

Preparing keywords column

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Sostituisci i valori NaN con una stringa vuota
df['keywords'] = df['keywords'].fillna('')

# Assicurati che tutti i valori nella colonna 'keywords' siano stringhe
df['keywords'] = df['keywords'].astype(str)

# Converti le stringhe di parole chiave in liste di parole chiave
df['keywords_list'] = df['keywords'].apply(lambda x: x.split(',') if x else [])

# Inizializza il MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Adatta il MultiLabelBinarizer alle liste di parole chiave e trasformale in vettori binari
labels = mlb.fit_transform(df['keywords_list'])

# Ora labels è un array binario che rappresenta la presenza/assenza di ciascuna parola chiave
# Si possono usare queste etichette per addestrare BERT

# Per vedere a quali parole chiave corrispondono le colonne in labels
print(mlb.classes_)

In [ ]:
# Converti l'array NumPy labels in un tensore PyTorch
labels_tensor = torch.tensor(labels, dtype=torch.float32)

Preparing body column

In [ ]:
from transformers import BertTokenizer

# Carica il tokenizzatore di BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Funzione per tokenizzare un testo con BERT
def tokenize_with_bert(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,  # Aggiungi '[CLS]' e '[SEP]'
        max_length=512,  # Imposta la massima lunghezza dei token
        padding='max_length',  # Aggiungi padding per raggiungere la massima lunghezza
        truncation=True,  # Tronca i token in eccesso
        return_attention_mask=True,  # Restituisci la maschera di attenzione
        return_tensors='pt'  # Restituisci tensori PyTorch
    )

# Applica la tokenizzazione al corpo combinato degli articoli
df['bert_input_body'] = df['combined_body'].apply(lambda x: tokenize_with_bert(x))

# Estrai i token e le maschere di attenzione per l'addestramento
input_ids_body = torch.cat([item['input_ids'] for item in df['bert_input_body']])
attention_masks_body = torch.cat([item['attention_mask'] for item in df['bert_input_body']])

Controlliamo il dataframe per vedere che non ci siano anomalie

In [ ]:
print(input_ids_body.size())
print(attention_masks_body.size())
print(labels_tensor.size())

## Modelling

Ora che abbiamo preparato le etichette con MultiLabelBinarizer e tokenizzato il corpo dell'articolo nella colonna combined_body, il prossimo passo è strutturare questi dati in un formato che BERT possa utilizzare per l'addestramento.

Ciò implica la creazione di un dataset PyTorch con i tokenizzati input_ids, le attention_masks e le etichette binarizzate.

Seguiremo questi passaggi:

1. Creazione del dataset PyTorch (TensorDataset che combina input_ids_body, attention_masks_body, labels)
2. Suddivisione training e validation set
3. Creazione dei DataLoader
4. Caricamento e configurazione di BERT
5. Traning di BERT
6. Valutazione di BERT
7. Salvataggio del modello

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Crea il TensorDataset
dataset = TensorDataset(input_ids_body, attention_masks_body, labels_tensor)

# Suddividi il dataset in set di addestramento e validazione (90-10)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Crea il DataLoader per il set di addestramento
train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = 16
)

# Crea il DataLoader per il set di validazione
validation_dataloader = DataLoader(
    val_dataset,
    sampler = SequentialSampler(val_dataset),
    batch_size = 16
)

Ora che abbiamo i DataLoader pronti, carichiamo BERT e lo prepariamo per il training

In [ ]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW

# Carica il modello pre-addestrato BERT per la classificazione delle sequenze
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Usa la variante 'base' di BERT con un tokenizer non case-sensitive
    num_labels = len(mlb.classes_),  # Il numero di classi determinate da MultiLabelBinarizer
    output_attentions = False,  # Devo approfondire che roba è
    output_hidden_states = False,  # Devo approfondire che roba è
)

# Sposta il modello sul dispositivo GPU se disponibile, altrimenti sarà su CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definisci l'ottimizzatore (AdamW è un ottimizzatore classico per i modelli BERT)
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,  # Argomenti per l'ottimizzatore possono variare
                  eps = 1e-8)  # Epsilon per la stabilità numerica

In [ ]:
from torch.nn import BCEWithLogitsLoss

# Definisci la funzione di perdita per la classificazione multilabel
loss_fn = BCEWithLogitsLoss()

In [ ]:
# Check per capire che labels sia un array 2D con forma [numero di esempi, numero di classi]
print(labels_tensor.shape)

In [ ]:
# Numero di epoche di addestramento
epochs = 4

# Ciclo di addestramento per il numero specificato di epoche
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    # Passo di addestramento
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        # Estrai i dati dal batch e trasferiscili sul dispositivo corretto
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Azzera i gradienti esistenti
        model.zero_grad()

        # Esegui un passaggio in avanti (valuta il modello sull'input del batch)
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_mask, labels=b_labels)

        # Estrai la perdita dal risultato del modello
        loss = outputs.loss
        total_train_loss += loss.item()

        # Esegui il backpropagation per calcolare i gradienti
        loss.backward()

        # Aggiorna i pesi del modello
        optimizer.step()

    # Calcola la perdita media per l'epoca
    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f'Training loss: {avg_train_loss}')

    # Valutazione del modello in modalità valutazione
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in validation_dataloader:
        # Estrai i dati dal batch e trasferiscili sul dispositivo corretto
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Disabilita il calcolo dei gradienti per la valutazione
        with torch.no_grad():
            # Esegui un passaggio in avanti (valuta il modello sull'input del batch)
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_mask, labels=b_labels)

        # Estrai la perdita dal risultato del modello
        loss = outputs.loss
        total_eval_loss += loss.item()

    # Calcola la perdita media per il set di validazione
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print(f'Validation loss: {avg_val_loss}')

    # Qui si potrebbe procedere anche con il calcolo di altre metriche di valutazione, come l'accuratezza

# Salvataggio del modello fine-tuned
model_save_path = './'
model.save_pretrained(model_save_path)

I risultati mostrano che il training e la validazione sono stati completati per tutte e 4 le epoche e che la loss di training e di validazione è diminuita con ogni epoca. Questo è un segnale positivo che indica che il modello sta imparando dai dati di addestramento e migliorando le prestazioni su dati non visti durante la validazione.

In [ ]:
# Salva il modello addestrato e il tokenizer per poterli ricaricare più tardi
model_save_path = './model/'
tokenizer_save_path = './model/'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

## Evaluation

In [ ]:
# Prendi un batch di dati dal validation_dataloader e le relative predizioni
batch = next(iter(validation_dataloader))
b_input_ids = batch[0].to(device)
b_attention_mask = batch[1].to(device)
outputs = model(b_input_ids, attention_mask=b_attention_mask)
logits = outputs.logits.detach().cpu().numpy()
print("Logit esempio:", logits[0])  # Logit del primo esempio nel batch

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# Supponiamo di avere un DataLoader per il set di test chiamato 'test_dataloader'
# e il tuo modello è già stato caricato e spostato sul dispositivo appropriato

# Imposta il modello in modalità valutazione
model.eval()

# Inizializza liste per le verità di base e le previsioni
true_labels = []
predictions = []

# Disabilita il calcolo dei gradienti per la valutazione
with torch.no_grad():
    for batch in validation_dataloader:
        # Estrai i dati dal batch e trasferiscili sul dispositivo corretto
        b_input_ids = batch[0].to(device)
        b_attention_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Esegui un passaggio in avanti (valuta il modello sull'input del batch)
        outputs = model(b_input_ids, attention_mask=b_attention_mask)
        logits = outputs.logits

        # Sposta i logits e le etichette su CPU per il calcolo delle metriche
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Aggiungi le etichette e le previsioni alle liste
        true_labels.append(label_ids)
        predictions.append(logits)

# Assicurati che true_labels e predicted_labels siano formati correttamente
true_labels = np.vstack(true_labels)
predictions = np.vstack(predictions)

# Converte le previsioni in valori binari basati su una soglia
threshold = 0.5
predicted_labels = (predictions > threshold).astype(int)

# Calcola le metriche
precision = precision_score(true_labels, predicted_labels, average='micro', zero_division=0)
recall = recall_score(true_labels, predicted_labels, average='micro', zero_division=0)
f1 = f1_score(true_labels, predicted_labels, average='micro', zero_division=0)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

## Usage

In [ ]:
# Testo di esempio da classificare
test_text = "Ticks are one of the main problems in production units, mainly because they have become resistant to the chemicals used to control them. Several alternative methods to chemicals have been sought to control tick infestations in cattle, which are practical and friendly to the environment. In this work, we implement rotational grazing to combat ticks at the pasture level. We found that a 30-day rest period for pastures (without animals) is not enough to reduce the presence of ticks in animals but that a 45-day rest period does reduce the presence of ticks in cattle. These studies are critical since they would help cattle producers design better strategies that help reduce the use of chemical acaricides and the presence of chemicals in milk, meat, and the environment."

# Tokenizza il testo
inputs = tokenizer.encode_plus(
    test_text,
    add_special_tokens=True,
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Sposta i tensori sul dispositivo corretto
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Esegui il modello
model.eval()
with torch.no_grad():
    outputs = model(input_ids, token_type_ids=None, attention_mask=attention_mask)
    logits = outputs.logits

# Applica la soglia ai logit per ottenere le previsioni binarie
threshold = 0.5
predictions = (logits.sigmoid().cpu().numpy() > threshold).astype(int)

# Mappa le previsioni binarie alle etichette di testo
predicted_labels = mlb.inverse_transform(predictions)

print("Testo di esempio:", test_text)
print("Etichette predette:", predicted_labels)

In [ ]:
predicted_labels_binary = (predictions > threshold).astype(int)

predicted_labels = mlb.inverse_transform(predicted_labels_binary)

for label_set in predicted_labels:
    print(", ".join([label for label in label_set]))